In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('AviationData.csv', encoding='windows-1252')

In [3]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [4]:
df.shape

(88889, 31)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [6]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

In [7]:
cols_to_drop = ['Latitude', 'Longitude', 'Schedule','Air.carrier', 'FAR.Description','Registration.Number',
                'Airport.Name', 'Airport.Code', 'Accident.Number', 'Event.Id', 'Investigation.Type',
               'Number.of.Engines', 'Purpose.of.flight', 'Report.Status', 'Injury.Severity', 'Publication.Date']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

In [8]:
df.isna().sum()

Event.Date                    0
Location                     52
Country                     226
Aircraft.damage            3194
Aircraft.Category         56602
Make                         63
Model                        92
Amateur.Built               102
Engine.Type                7096
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
dtype: int64

In [9]:
# Impute numerical data
df['Total.Fatal.Injuries'].fillna(0, inplace=True)
df['Total.Serious.Injuries'].fillna(0, inplace=True)
df['Total.Minor.Injuries'].fillna(0, inplace=True)
df['Total.Uninjured'].fillna(0, inplace=True)

In [10]:
df['Total.Injured'] = df['Total.Fatal.Injuries']+df['Total.Serious.Injuries']+df['Total.Minor.Injuries']

In [11]:
cols_to_drop = ['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

In [12]:
# Fill missing values for categorical columns
df['Location'].fillna('Unknown', inplace=True)
df['Country'].fillna('Unknown', inplace=True)
df['Broad.phase.of.flight'].fillna('Unknown', inplace=True)
df['Weather.Condition'].fillna('Unknown', inplace=True)
df['Engine.Type'].fillna('Unknown', inplace=True)
df['Aircraft.Category'].fillna('Unknown', inplace=True)
df['Aircraft.damage'].fillna('Unknown', inplace=True)
df['Amateur.Built'].fillna('Unknown', inplace=True)

In [13]:
df['Make'].fillna(df['Make'].mode()[0], inplace=True)

In [14]:
df['Model'] = df.groupby('Make')['Model'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else 'Unknown')

In [15]:
df.isna().sum()

Event.Date               0
Location                 0
Country                  0
Aircraft.damage          0
Aircraft.Category        0
Make                     0
Model                    0
Amateur.Built            0
Engine.Type              0
Total.Uninjured          0
Weather.Condition        0
Broad.phase.of.flight    0
Total.Injured            0
dtype: int64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Event.Date             88889 non-null  object 
 1   Location               88889 non-null  object 
 2   Country                88889 non-null  object 
 3   Aircraft.damage        88889 non-null  object 
 4   Aircraft.Category      88889 non-null  object 
 5   Make                   88889 non-null  object 
 6   Model                  88889 non-null  object 
 7   Amateur.Built          88889 non-null  object 
 8   Engine.Type            88889 non-null  object 
 9   Total.Uninjured        88889 non-null  float64
 10  Weather.Condition      88889 non-null  object 
 11  Broad.phase.of.flight  88889 non-null  object 
 12  Total.Injured          88889 non-null  float64
dtypes: float64(2), object(11)
memory usage: 8.8+ MB


In [38]:
df['Total.Uninjured'].unique()

array([  0.,  44.,   2.,   1.,   3.,   6.,   4., 149.,  12., 182., 154.,
         5.,  10.,   7., 119.,  36.,  51.,  16.,  83.,   9.,  68.,  30.,
        20.,  18.,   8., 108.,  11., 152.,  21.,  48.,  56., 113., 129.,
       109.,  29.,  13.,  84.,  74., 142., 102., 393., 128., 112.,  17.,
        65.,  67., 136.,  23., 116.,  22.,  57.,  58.,  73., 203.,  31.,
       201., 412., 159.,  39., 186., 588.,  82.,  95., 146., 190., 245.,
       172.,  52.,  25.,  59., 131., 151., 180., 150.,  86.,  19., 133.,
       240.,  15., 145., 125., 440.,  77., 122., 205., 289., 110.,  79.,
        66.,  87.,  78.,  49., 104., 250.,  33., 138., 100.,  53., 158.,
       127., 160., 260.,  47.,  38., 165., 495.,  81.,  41.,  14.,  72.,
        98., 263., 188., 239.,  27., 105., 111., 212., 157.,  46., 121.,
        75.,  71.,  45.,  91.,  99.,  85.,  96.,  50.,  93., 276., 365.,
       371., 200., 103., 189.,  37., 107.,  61.,  26., 271., 130.,  89.,
       439., 132., 219.,  43., 238., 195., 118., 17

In [36]:
df.describe()

,Total.Uninjured,Total.Injured
count,88889.000000,88889.000000
mean,4.971245,1.114379
std,27.002011,6.027319
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,1.000000
max,699.000000,380.000000


In [40]:
# Save the cleaned dataset
df.to_csv('Cleaned_AviationData.csv')